https://github.com/PacktPublishing/Python-Social-Media-Analytics/blob/5886b29587054526920e40b54ea630de5ed9d365/Chapter03/Chapter3.ipynb


https://developers.facebook.com/docs/graph-api/explorer/

https://developers.facebook.com/docs/places/search

https://developers.facebook.com/docs/graph-api/reference/page/ratings/

https://developers.facebook.com/tools/explorer/

In [3]:
#!pip install --user facepy
#!pip install pony

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import requests
import json
import time
import db_reacao
from datetime import datetime
from facepy import GraphAPI
from pony import orm

In [2]:
def insert_facebook(cliente, page_id, query_time_post, query_time_comm, get_feed,access_token=False):
    global erros_insert_comment
    global insert_comment
    min_datetime_target = False
    
    post={}
    comm={}
    reac={}
    reac_sentimento = {"SAD":-1,"ANGRY":-1,"HAHA":1,"LOVE":1,"WOW":1,"LIKE":1}
    
    if get_feed==True:
        
        if access_token==False:
            str_url_facebook = 'v2.12/' + page_id + '/feed?'
        else:
            str_url_facebook = 'v2.12/' + page_id.split("v2.12/")[1].split("/feed?access_token=")[0] + '/feed?access_token=' + page_id.split("v2.12/")[1].split("/feed?access_token=")[1]
        
        data_post = graph.get(str_url_facebook)
        for p in range(0,len(data_post["data"])):
            post = data_post["data"][p]
            created_time_post = datetime.strptime(post.get("created_time").split("T")[0], '%Y-%m-%d')
            if created_time_post>=query_time_post:
                data_comm = graph.get('v2.12/'+ post.get("id") +'?fields=comments,reactions')
                if data_comm.get("comments")!=None:                
                    for c in range(0,len(data_comm["comments"]["data"])):
                        comm = data_comm["comments"]["data"][c]
                        created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                        if created_time_comm>=query_time_comm:
                            try:
                                insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                                db_reacao.add_reacao_face(cliente,2,comm["message"], 0,str(created_time_comm),str(comm["id"]))
                                #db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                                print("Registro inserido".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                insert_comment += 1
                            except Exception as e:
                                erros_insert_comment += 1
                                print("Registro com erro".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                print("Erro: " + str(e))  
                if data_comm.get("reactions")!=None:
                    for c in range(0,len(data_comm["reactions"]["data"])):
                        reac = data_comm["reactions"]["data"][c]
                        try:            
                            db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),str(datetime.now()),str(post["id"])+"_"+str(reac["id"]))
                            #db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                            print("Registro inserido".center(110,"*"))
                            print("Reação: " + reac["type"])
                            insert_comment += 1
                        except Exception as e:
                            erros_insert_comment += 1
                            print("Registro com erro".center(110,"*"))
                            print("Reação: " + reac["type"])
                            print("Erro: " + str(e))    
            else:
                min_datetime_target=True
                
        if min_datetime_target==False:
            if data_post.get('paging')!=None:
                next_page = data_post.get('paging')
                if next_page.get('next')!=None:
                    next_page_id = next_page.get('next')
                    insert_facebook(cliente, next_page_id, query_time_post, query_time_comm,get_feed,True)
            
    else:
        data_comm = graph.get('v2.12/'+ page_id +'?fields=comments,reactions')        
        if data_comm.get("comments")!=None:                
            for c in range(0,len(data_comm["comments"]["data"])):
                comm = data_comm["comments"]["data"][c]
                created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                if created_time_comm>=query_time_comm:
                    try:
                        insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                        db_reacao.add_reacao_face(cliente,2,comm["message"], 0,str(created_time_comm),str(comm["id"]))
                        #db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                        print("Registro inserido".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        insert_comment += 1
                    except Exception as e:
                        erros_insert_comment += 1
                        print("Registro com erro".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        print("Erro: " + str(e))  
        if data_comm.get("reactions")!=None:
            for c in range(0,len(data_comm["reactions"]["data"])):
                reac = data_comm["reactions"]["data"][c]
                try:            
                    db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),str(datetime.now()),str(post["id"])+"_"+str(reac["id"]))
                    #db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                    print("Registro inserido".center(110,"*"))
                    print("Reação: " + reac["type"])
                    insert_comment += 1
                except Exception as e:
                    erros_insert_comment += 1
                    print("Registro com erro".center(110,"*"))
                    print("Reação: " + reac["type"])
                    print("Erro: " + str(e))    

In [3]:
# Initialize the Graph API with a valid access token (optional,
# but will allow you to do all sorts of fun stuff).
graph = GraphAPI("EAACEdEose0cBABktlXZBpi7JHRXPX9OrhXiLpyJyWO5QJfYutjGN1BEcKny2WOWcJZAH34ssZBI8i4dNQTy7e9UvniZC1I0AHxUTQEiDgFvO4BIvKxqZCdyjcDL6RR1gzFPYMppFzsSvKpbLi3I6zZB8FC8l9IkmgICZCUfiPe96VbkXrRHOgcZBvGWejFdXXj8ZD")

In [4]:
query_time_post = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_comm = datetime.strptime("2018-05-01", '%Y-%m-%d')

In [5]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '1'
page_id = 'casariobotequim'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 0
Erros de inserção de comentario: 0


In [6]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '4'
page_id = 'casariobeach'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 0
Erros de inserção de comentario: 0


In [7]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '5'
page_id = 'vereadorlelecooficial'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 0
Erros de inserção de comentario: 0


In [8]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '7'
page_id = 'globoformula'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: Olá, Kelly :D
Respondemos você pelo menssenger <3
**********************************************Registro inserido***********************************************
Comentário: Valor?
**********************************************Registro inserido***********************************************
Comentário: Ei, Jessica :D
Respondemos você pelo menssenger <3
**********************************************Registro com erro***********************************************
Comentário: preço por favor?
Erro: Object Reacao[new:4] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro inserido***********************************************
Comentário: Olá, Isabel! Enviamos o orçamento por inbox :D
**********************************************Registro com erro********************

In [9]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '2'
page_id = 'oleodealegriavv'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: Olha Anna Mombrini
**********************************************Registro inserido***********************************************
Comentário: boa noite sim! taxa 5 reais
**********************************************Registro inserido***********************************************
Comentário: Boa noite
Entrega em alvorada?
**********************************************Registro com erro***********************************************
Comentário: Precisamos marca de ir Anderson Manoel.
Erro: Object Reacao[new:16] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: 
Erro: Attribute Reacao.reacao is required
**********************************************Registro com erro******************************

**********************************************Registro com erro***********************************************
Comentário: Sexta dia 11/05 estarei lá. Bora pro óleo meninas???
Erro: Object Reacao[new:45] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Aleluiaaa....
Erro: Object Reacao[new:46] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Estou dentro
Erro: Object Reacao[new:47] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Aí Rafaela Rodrigues....foi desse l

**********************************************Registro com erro***********************************************
Comentário: Célia Filardy Mezrahy Paula Duarte Mel Marlene Vanessa Cristina Bruna Panar a próxima vez que vcs vierem, vamos lá 😂😂😂
Erro: Object Reacao[new:76] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: 
Erro: Attribute Reacao.reacao is required
**********************************************Registro com erro***********************************************
Comentário: Isabela Leite Tayany Lourenço hahahahahaha
Vou fazer sucesso la rsrsrsrsrs
Erro: Object Reacao[new:77] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Taty Olyv

In [10]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '8'
page_id = 'VilaBoliche'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: http://vilaparqueboliche.com.br/
*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 1
Erros de inserção de comentario: 0
